# Convert ply to raster

In [18]:
library("raster")
library("maptools")
library("data.table")

library("lidR")

library("e1071")


Attaching package: 'e1071'

The following object is masked from 'package:raster':

    interpolate



In [3]:
# global variables

workingdirectory="D:/Koma/escience/test_escience_2018Nov/exampledata/"
filename="tile_00005_subset"

setwd(workingdirectory)

In [10]:
start_time <- Sys.time()

pc_fea = fread(paste(filename ,"_cell_1m.ply",sep=""), header = F, sep = ' ',skip=31)
#print(head(pc_fea))

pc_fea_var_z <- rasterFromXYZ(pc_fea[,c(1,2,18)])
writeRaster(pc_fea_var_z, paste(filename,"_var_z.tif",sep=""),overwrite=TRUE)

end_time <- Sys.time()
print(end_time - start_time)

Time difference of 0.117882 secs


# Same with LiDR

In [21]:
start_time <- Sys.time()

las = readLAS(paste(filename,".las",sep=""))

HeightMetrics = function(z)
{
  heightmetrics = list(
    zmax = max(z), 
    zmean = mean(z),
    zmedian = median(z),
    zmin = min(z),
    zstd = sd(z),
    zvar = var(z),
    zcoeffvar = sd(z)/mean(z),
    zskew = skewness(z),
    zkurto = kurtosis(z)
  )
  return(heightmetrics)
}

ShapeMetrics = function(X,Y,Z)
{
  xyz=rbind(X,Y,Z) 
  cov_m=cov(xyz)
  eigen_m=eigen(cov_m)
  
  shapemetrics = list(
    eigen_largest = eigen_m$values[1],
    eigen_medium = eigen_m$values[2],
    eigen_smallest = eigen_m$values[3]
  )
  return(shapemetrics)
}

heightmetrics = grid_metrics(las, HeightMetrics(Z),res=1)
shapemetrics = grid_metrics(las, ShapeMetrics(X,Y,Z),res=1)

var_z_r <- rasterFromXYZ(heightmetrics[,c(1,2,6)])
writeRaster(var_z_r, paste(filename,"_var_z_LiDR.tif",sep=""),overwrite=TRUE)

end_time <- Sys.time()
print(end_time - start_time)

169886 points below 0 found.
127813 unclassified points found.
Warning message:
"Invalid file: some points are outside the bounding box defined by the header"Warning message:
"Invalid file: some points are outside the elevation range defined by the header"

Time difference of 11.8624 secs
